# Meander Maker

If you haven't ever used the `googlemaps` python package, there are a few small setup steps to take first. You'll need to install it by running the next cell. Additionally, we will use `gmplot` to render the outputs and `polyline` to decode [Google's "Polyline"](https://developers.google.com/maps/documentation/utilities/polylineutility) hash. Finally, generate an API key to interface with Google's services. (There is a free tier, but if you use it a lot, they will start charging you for access.)

In [ ]:
!pip install googlemaps
!pip install gmplot
!pip install polyline

If you would like to tinker around with the code, you'll need your own API key for Google Maps. You can register one from this address: [https://developers.google.com/places/web-service/get-api-key](https://developers.google.com/places/web-service/get-api-key). Once you have one, create a file in the directory containing this notebook called `.secret.key` and simply paste in your newly generated API key, no quotes or line breaks or anything. Save, exit, and you should be good to go.

In [117]:
import pandas as pd
import numpy as np
import googlemaps
import polyline
from gmplot import gmplot
import goplaces as gp

%load_ext autoreload
%autoreload 2

with open('.secret.key') as f:
    api_key = f.read().strip()

gmaps = googlemaps.Client(key=api_key)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Let's decide which topic to explore:

In [138]:
topic = gp.get_topic()

What theme walk would you like to explore today? wine tasting


In [139]:
loc = gp.get_loc(current=False)

Where would you like to start? Napa, CA


In [140]:
one_way_json = gp.build_list(loc, topic)
dest_list = gp.lat_lng_list(one_way_json)
walk = gp.walk(dest_list, verbose=False)

In [141]:
gp.html_builder(loc, walk)

---------TESTING-----------

In [127]:
df = pd.DataFrame([pivot['end_location'] for pivot in walk['legs']])
poly = np.array(
    polyline.decode(walk['overview_polyline']['points'])
)

In [133]:
gmapit = gmplot.GoogleMapPlotter(loc['lat'], loc['lng'], zoom=17, apikey=api_key)

for lat, lng in df.values:
    gmapit.marker(lat, lng, 'cornflowerblue', 
                  title=f"{round(lat, 3)}, {round(lng, 3)}")

gmapit.scatter(df['lat'], df['lng'], '#f542a1', size=8, marker=False)
gmapit.plot(poly[:,0], poly[:,1])
gmapit.draw("mymap.html")